## Dimension Reduction
### 차원의 저주
차원이 증가할 수록 데이터가 sparse 해진다.

### 차원 축소
1. 투영 : 모든 훈련 샘플이 고차원 공간 안의 저차원 부분 공간에 놓여 있다고 가정
한계 : 스위스 롤 (롤 케이크 차원) 처럼 언제나 투영이 최선은 아님


2. 매니폴드 : 고차원 공간에서 휘어지거나 뒤틀린 모양
매니폴드 학습 : 매니폴드 가정(고차원 데이터셋이 저차원 매니폴드에 가깝게 놓여 있음을 가정) 
한계 : 저차원 매니폴드 공간이 원본보다 더 간단하게 표현된다고 가정하는데, 항상 유효하지 않음


3. PCA : 주성분 분석
데이터에 가까운 초평면을 정의하고 이 평면에 투영


### 적절한 차원 수 선택하기
데이터 시각화를 위한 차원 축소는 2-3차원이 적당하나 훈련 세트의 분산이 95%로 유지하는데 적절한 차원 수를 선택하는 것이 간단하다.

1. 분산 이용하는 방법

분산이 어느정도(0.95)로 유지되는 데 필요한 최소한의 차원 수를 계산해 설정

2. 분산을 차원에 대한 함수로 나타내는 방법

cumsum을 그래프로 그려 분산의 성장이 멈추는 변곡점을 알아냄

In [2]:
import numpy as np
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

/Users/seyeong/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95 ) + 1

In [5]:
d

154

In [6]:
# pca = PCA(n_components=d)
pca = PCA(n_components=0.95) # 위와 같다
X_reduced = pca.fit_transform(X_train)

### 압축을 위한 PCA
투영을 통해 차원을 압축한다. 이를 다시 재구성하여 원본 차원으로 비슷하게 복원할 수 있다. 
(5%의 손실에 대해 복원할 수 없음)

- 재구성 오차 : 원본 데이터 - 재구성된 데이터 사이의 평균 제곱 거리

### 랜덤 PCA
확률적 알고리즘을 사용해 처음 d개의 주성분에 대한 근삿값을 빠르게 찾는다. 축소되는 차원 d가 n보다 매우 작다면 SVD보다 매우 빠르다.


In [7]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)